In [ ]:
import os
import cv2
import numpy as np
from imutils import face_utils
import dlib
import matplotlib.pyplot as plt

In [ ]:
input_folder = "archive/train"
processed_folder = "D:\\2. College project\\sem 5 drowsiness detection while driving\\driver_drowsiness_system_CNN\\new_test_processing_1"
predictor = dlib.shape_predictor('data/shape_predictor_68_face_landmarks.dat')
face_cascade = cv2.CascadeClassifier('data/haarcascade_frontalface_default.xml')

# Image Processing

In [ ]:
def apply_sharpening(image):
    kernel = np.array([[0, -1, 0], [-1, 5, -1], [0, -1, 0]])
    return cv2.filter2D(image, -1, kernel)

def remove_noise_gaussian(image, kernel_size=(3, 3)):
    return cv2.GaussianBlur(image, kernel_size, 0)

def remove_noise_bilateral(image, diameter=9, sigma_color=75, sigma_space=75):
    return cv2.bilateralFilter(image, diameter, sigma_color, sigma_space)

def normalize_image(image):
    return image / 255.0

In [ ]:
def preprocess_eye_image(image_path):
    img = cv2.imread(image_path)
    if img is None:
        print(f"Failed to read image: {image_path}")
        return None

    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    denoised_img = remove_noise_bilateral(img)

    sharpened_img = apply_sharpening(denoised_img)

    resized_img = cv2.resize(sharpened_img, (224, 224), interpolation=cv2.INTER_CUBIC)
    normalized_img = normalize_image(resized_img)

    return normalized_img

In [ ]:
def preprocess_yawn_image(image_path):
    img = cv2.imread(image_path)
    if img is None:
        print(f"Failed to read image: {image_path}")
        return None

    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    detector = dlib.get_frontal_face_detector()
    faces = detector(img, 1)
    if len(faces) == 0:
        print(f"No face detected in image: {image_path}")
        return None

    for face in faces:
        landmarks = predictor(img, face)
        landmarks = face_utils.shape_to_np(landmarks)

        mouth_points = landmarks[48:68]
        (x_min, y_min) = mouth_points.min(axis=0)
        (x_max, y_max) = mouth_points.max(axis=0)

        margin = 15
        x_min = max(0, x_min - margin)
        y_min = max(0, y_min - margin)
        x_max = min(img.shape[1], x_max + margin)
        y_max = min(img.shape[0], y_max + margin)

        cropped_img = img[y_min:y_max, x_min:x_max]

        denoised_img = remove_noise_bilateral(cropped_img)

        sharpened_img = apply_sharpening(denoised_img)

        resized_img = cv2.resize(sharpened_img, (224, 224), interpolation=cv2.INTER_CUBIC)
        normalized_img = normalize_image(resized_img)

        return normalized_img

In [ ]:
def plot_processing_steps(original_img, steps, title_steps):
    plt.figure(figsize=(15, 5))
    plt.subplot(1, len(steps) + 1, 1)
    plt.imshow(original_img)
    plt.title("Original")
    plt.axis('off')

    for i, (step_img, title) in enumerate(zip(steps, title_steps)):
        plt.subplot(1, len(steps) + 1, i + 2)
        plt.imshow(step_img)
        plt.title(title)
        plt.axis('off')
    
    plt.tight_layout()
    plt.show()

def preprocess_with_plot(image_path, category):
    img = cv2.imread(image_path)
    if img is None:
        print(f"Failed to read image: {image_path}")
        return

    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    steps = [img_rgb]
    titles = ["Original"]

    if category in ['Closed', 'Open']:
        denoised_img = remove_noise_bilateral(img_rgb)
        sharpened_img = apply_sharpening(denoised_img)
        resized_img = cv2.resize(sharpened_img, (224, 224), interpolation=cv2.INTER_CUBIC)
        normalized_img = normalize_image(resized_img)

        steps.extend([denoised_img, sharpened_img, resized_img, normalized_img])
        titles.extend(["Denoised", "Sharpened", "Resized", "Normalized"])

    elif category in ['yawn', 'no_yawn']:
        detector = dlib.get_frontal_face_detector()
        faces = detector(img_rgb, 1)
        if len(faces) == 0:
            print(f"No face detected in image: {image_path}")
            return

        for face in faces:
            landmarks = predictor(img_rgb, face)
            landmarks = face_utils.shape_to_np(landmarks)

            mouth_points = landmarks[48:68]
            (x_min, y_min) = mouth_points.min(axis=0)
            (x_max, y_max) = mouth_points.max(axis=0)

            margin = 15
            x_min = max(0, x_min - margin)
            y_min = max(0, y_min - margin)
            x_max = min(img_rgb.shape[1], x_max + margin)
            y_max = min(img_rgb.shape[0], y_max + margin)

            cropped_img = img_rgb[y_min:y_max, x_min:x_max]

            denoised_img = remove_noise_bilateral(cropped_img)
            sharpened_img = apply_sharpening(denoised_img)
            resized_img = cv2.resize(sharpened_img, (224, 224), interpolation=cv2.INTER_CUBIC)
            normalized_img = normalize_image(resized_img)

            steps.extend([cropped_img, denoised_img, sharpened_img, resized_img, normalized_img])
            titles.extend(["Mouth ROI", "Denoised", "Sharpened", "Resized", "Normalized"])
            break  

    plot_processing_steps(img_rgb, steps, titles)

def visualize_selected_images(categories):
    for category, image_path in categories.items():
        print(f"Visualizing processing for category: {category}")
        preprocess_with_plot(image_path, category)

categories = {
    'Closed': "archive/train/Closed/_0.jpg",
    'Open': "archive/train/Open/_10.jpg",
    'yawn': "archive/train/yawn/7.jpg",
    'no_yawn': "archive/train/no_yawn/3.jpg"
}

visualize_selected_images(categories)

In [ ]:
def process_and_save_images(input_folder, processed_folder):
    if not os.path.exists(processed_folder):
        os.makedirs(processed_folder)

    for category in os.listdir(input_folder):
        category_path = os.path.join(input_folder, category)
        output_category_folder = os.path.join(processed_folder, category)
        if not os.path.exists(output_category_folder):
            os.makedirs(output_category_folder)

        for filename in os.listdir(category_path):
            image_path = os.path.join(category_path, filename)

            if category in ['yawn', 'no_yawn']:
                processed_image = preprocess_yawn_image(image_path)
            elif category in ['Closed', 'Open']:
                processed_image = preprocess_eye_image(image_path)
            else:
                print(f"Skipped unknown category: {category}")
                continue

            if processed_image is not None:
                processed_image = (processed_image * 255).astype(np.uint8)
                output_path = os.path.join(output_category_folder, filename)
                cv2.imwrite(output_path, cv2.cvtColor(processed_image, cv2.COLOR_RGB2BGR))
                print(f"Processed and saved: {output_path}")

process_and_save_images(input_folder, processed_folder)